In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import numpy as np
import math

tasks = open("Participante_2/p2_ts_1582222326745.json", "r") 
json_tasks = tasks.read()
tasks_data = json.loads(json_tasks)
df = pd.DataFrame(tasks_data)

df.to_csv("df.csv", index=False)

plt.figure(figsize=(8.64, 6))

for i in range(len(df.tail(500).posX )):
    posX = df.loc[i,"posX"]
    posY = `df.loc[i,"posY"]*-1
    
    plt.plot(posX, posY, "o", c="b", markersize=7, alpha=0.5)
plt.xlim([0,1080])
plt.ylim([-750,0])

df = df[(df["time"]> 10*1000) & (df["time"]< 110*1000)]


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
def get_df(file):
    tasks = open(file, "r") 
    json_tasks = tasks.read()
    tasks_data = json.loads(json_tasks)
    df = pd.DataFrame(tasks_data)
    df["new_posX"] = list(df.posX.iloc[1:])+[0]
    df["new_posY"] = list(df.posY.iloc[1:])+[0]
    df["mov_X"] = (df["new_posX"] -  df["posX"] )
    df["mov_Y"] = (df["new_posY"] -  df["posY"] )
    df["distance"] = (df.mov_X**2 + df.mov_Y**2)**(1/2)
    df["angle"] = df.apply(lambda r: math.atan2(r.mov_Y, r.mov_X) * (180.0 / math.pi),axis=1)
    return df

In [0]:
def get_distance(x1,y1,x2=540,y2=360):
    return ((x1 - x2)**2+(y1 - y2)**2)**(1/2)

In [0]:
def get_radius(df):
    return df[["posX","posY"]].apply(lambda r: get_distance(r["posX"],r["posY"]), axis=1).mean()

In [0]:
def get_dinamic_radius(df, i=6):
    x = df.copy()
    x["time_interval"] = pd.cut(x["time"],range(0,120001,int(120000/i)),labels=list(range(1,i+1)))
    x.dropna(inplace=True)
    x["dinamic_center_X"] = x[["time_interval","posX"]].groupby('time_interval').transform('mean')
    x["dinamic_center_Y"] = x[["time_interval","posY"]].groupby('time_interval').transform('mean')
    return x[["posX","posY","dinamic_center_X", "dinamic_center_Y"]].apply(lambda r: get_distance(r["posX"],r["posY"], r["dinamic_center_X"], r["dinamic_center_Y"]), axis=1).mean()

def get_radius_variance_X(df):
    return df["dinamic_center_X"].unique().std()
def get_radius_variance_Y(df):
    return df["dinamic_center_Y"].unique().std()

In [0]:
def touch_area_variance(df):
    return (df["height"] * df["width"]).std()

In [0]:
def get_rounds_s_fseconds(df, s):
    x = df[df.time<s*1000][["posX","posY","distance"]]
    center= x.posX.mean() , x.posY.mean()
    x["radius"] = x.apply(lambda r: get_distance(r.posX, r.posY, center[0], center[1]), axis=1)
    radius = x.radius.mean()
    return x.distance.sum()/(2*math.pi*radius)


In [0]:
def get_rounds_s_lseconds(df, s):
    x = df[df.time>120000-s*1000][["posX","posY","distance"]]
    center= x.posX.mean() , x.posY.mean()
    x["radius"] = x.apply(lambda r: get_distance(r.posX, r.posY, center[0], center[1]), axis=1)
    radius = x.radius.mean()
    return x.distance.sum()/(2*math.pi*radius)


In [0]:
def get_displacement_center_s_seconds(df, s):
    x = df[df.time<s*1000][["posX","posY","distance"]]
    return get_distance(x.posX.mean() , x.posY.mean())


In [0]:
def get_avg_distance_between_points(df):
    return df.distance.mean()
def get_std_distance_between_points(df):
    return df.distance.std()

In [0]:
def generate_row(df,cat,dif,n,turn, p):
    dic = {"participant_id":[p],"category":[cat], "difficulty":[dif], "order":[n], "turn":[turn],  "radius":[], 
           "dynamic_radius_30s":[], "dynamic_radius_20s":[],"dynamic_radius_15s":[], "dynamic_radius_10s":[], "dynamic_radius_5s":[],
           "rounds_10_fseconds":[], "rounds_15_fseconds":[], "rounds_30_fseconds":[], "rounds_60_fseconds":[],
           "rounds_10_lseconds":[], "rounds_15_lseconds":[], "rounds_30_lseconds":[], "rounds_60_lseconds":[],
           "avg_distance_between_points":[],"std_distance_between_points":[],
           "distance_center_10_seconds":[] ,"distance_center_15_seconds":[], "distance_center_30_seconds":[], "distance_center_60_seconds":[] }
    dic["radius"].append(get_radius(df))
    dic["dynamic_radius_30s"].append(get_dinamic_radius(df,4))
    dic["dynamic_radius_20s"].append(get_dinamic_radius(df,6))
    dic["dynamic_radius_15s"].append(get_dinamic_radius(df,8))
    dic["dynamic_radius_10s"].append(get_dinamic_radius(df,12))
    dic["dynamic_radius_5s"].append(get_dinamic_radius(df,24))
    
    dic["rounds_10_fseconds"].append(get_rounds_s_fseconds(df,10))
    dic["rounds_15_fseconds"].append(get_rounds_s_fseconds(df,15))
    dic["rounds_30_fseconds"].append(get_rounds_s_fseconds(df,30))
    dic["rounds_60_fseconds"].append(get_rounds_s_fseconds(df,60))
    
    dic["rounds_10_lseconds"].append(get_rounds_s_lseconds(df,10))
    dic["rounds_15_lseconds"].append(get_rounds_s_lseconds(df,15))
    dic["rounds_30_lseconds"].append(get_rounds_s_lseconds(df,30))
    dic["rounds_60_lseconds"].append(get_rounds_s_lseconds(df,60))
    
    
    dic["distance_center_10_seconds"].append(get_displacement_center_s_seconds(df,10))
    dic["distance_center_15_seconds"].append(get_displacement_center_s_seconds(df,15))
    dic["distance_center_30_seconds"].append(get_displacement_center_s_seconds(df,30))
    dic["distance_center_60_seconds"].append(get_displacement_center_s_seconds(df,60))
           
               
    dic["avg_distance_between_points"].append(get_avg_distance_between_points(df))
    dic["std_distance_between_points"].append(get_std_distance_between_points(df))

    return pd.DataFrame(dic)
    

In [0]:
root= "drive/My Drive/CognitiveLoad/Experimentos/"

In [0]:
participante = "Participante_1"
file = "1_numeric_hard.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
turn = "R"
data = generate_row(df, cat, dif, n, turn, "p1")

participante = "Participante_1"
file = "1_numeric_hard.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
turn = "R"
data = generate_row(df, cat, dif, n, turn, "p1")

file = "3_sequence_hard.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p1")])

file = "4_sequence_easy.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p1")])

file = "5_verbal_hard.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p1")])

file = "6_verbal_easy.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p1")])


In [0]:
participante = "Participante_2"
file = "1_numeric_hard.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
turn = "L"
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p2")])

file = "2_numeric_easy.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p2")])

file = "3_sequence_easy.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p2")])

file = "4_sequence_hard.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p2")])

file = "5_verbal_easy.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p2")])

file = "6_verbal_hard.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p2")])


In [39]:
participante = "Participante_3"
file = "1_numeric_easy.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
turn = "L"
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p3")])

file = "2_numeric_hard.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p3")])

file = "3_sequence_hard.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p3")])

file = "4_sequence_easy.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p3")])

file = "5_verbal_easy.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p3")])

file = "6_verbal_hard.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p3")])


ValueError: ignored

In [0]:
participante = "Participante_4"
file = "1_numeric_easy.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
turn = "R"
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p4")])

file = "2_numeric_hard.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p4")])

file = "3_sequence_hard.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p4")])

file = "4_sequence_easy.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p4")])

file = "5_verbal_hard.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p4")])

file = "6_verbal_easy.json"
df = get_df(root+participante+"/"+ file)
n, cat, dif= file.split(".")[0].split("_")
data = pd.concat([data,generate_row(df, cat, dif, n, turn, "p4")])


In [42]:
data.tail()

,participant_id,category,difficulty,order,turn,radius,dynamic_radius_30s,dynamic_radius_20s,dynamic_radius_15s,dynamic_radius_10s,dynamic_radius_5s,rounds_10_fseconds,rounds_15_fseconds,rounds_30_fseconds,rounds_60_fseconds,rounds_10_lseconds,rounds_15_lseconds,rounds_30_lseconds,rounds_60_lseconds,avg_distance_between_points,std_distance_between_points,distance_center_10_seconds,distance_center_15_seconds,distance_center_30_seconds,distance_center_60_seconds
0,p4,numeric,hard,2,R,150.657086,127.638723,127.045847,127.057699,126.552078,126.248785,8.628847,12.933558,25.379174,51.943841,14.353714,18.378385,30.764942,55.976493,11.456027,9.339906,36.169791,43.011021,53.572038,69.087934
0,p4,sequence,hard,3,R,176.430844,130.262994,130.026443,129.835282,129.421330,128.791835,8.297109,12.623420,28.274903,51.807936,15.718741,19.957450,33.954525,62.048429,12.637772,16.051158,57.768943,58.838447,65.514676,119.991024
0,p4,sequence,easy,4,R,136.669195,132.188463,132.189713,131.639276,131.518787,130.617144,11.163417,16.176049,31.387655,61.705012,13.419548,18.216961,30.806529,58.179672,13.233925,11.472495,34.937465,35.098266,51.837995,48.807533
0,p4,verbal,hard,5,R,227.058906,122.197526,120.033198,119.863808,118.836782,117.609088,8.935082,13.750480,29.716047,55.072953,16.798523,21.674015,36.242672,62.883967,12.341549,10.009408,21.502035,32.355032,61.648026,122.599273
0,p4,verbal,easy,6,R,178.395031,109.295389,109.161763,108.510787,108.165150,107.558981,12.018013,17.643046,32.356887,58.382540,22.180329,26.954225,42.814357,74.861796,12.261963,13.763514,37.119290,44.689750,72.360771,113.259662


In [0]:
len(data.columns)

21

In [0]:
data.columns

Index(['participant_id', 'category', 'difficulty', 'order', 'turn', 'radius',
       'dynamic_radius_30s', 'dynamic_radius_20s', 'dynamic_radius_15s',
       'dynamic_radius_10s', 'dynamic_radius_5s', 'rounds_10_fseconds',
       'rounds_15_fseconds', 'rounds_30_fseconds', 'rounds_60_fseconds',
       'rounds_10_lseconds', 'rounds_15_lseconds', 'rounds_30_lseconds',
       'rounds_60_lseconds', 'avg_distance_between_points',
       'std_distance_between_points', 'displacement_center_10_seconds',
       'displacement_center_15_seconds', 'displacement_center_30_seconds',
       'displacement_center_60_seconds'],
      dtype='object')

In [0]:
data

,participant_id,category,difficulty,order,turn,radius,dynamic_radius_30s,dynamic_radius_20s,dynamic_radius_15s,dynamic_radius_10s,...,rounds_10_lseconds,rounds_15_lseconds,rounds_30_lseconds,rounds_60_lseconds,avg_distance_between_points,std_distance_between_points,displacement_center_10_seconds,displacement_center_15_seconds,displacement_center_30_seconds,displacement_center_60_seconds
0,p1,numeric,hard,1,R,218.320752,171.381115,170.996476,170.543908,170.397529,...,18.328569,23.988042,40.261814,72.702734,20.613820,26.978427,29.008719,36.234571,79.219572,120.338036
0,p1,sequence,hard,3,R,210.702228,151.504307,151.275239,151.099553,149.963898,...,20.238359,26.614507,46.035443,83.447831,20.045792,16.935389,79.384541,115.020906,147.432271,166.027148
0,p1,sequence,easy,4,R,180.355632,148.095118,147.350803,145.933765,146.252666,...,19.194341,25.454835,44.829843,85.198859,19.767631,10.700135,65.433492,79.246004,91.603968,127.479601
0,p1,verbal,hard,5,R,172.418048,165.770969,165.787067,165.580939,165.501016,...,20.514246,26.817826,46.110588,84.151548,20.745179,10.707033,51.415479,62.738578,59.494791,59.026707
0,p1,verbal,easy,6,R,195.785280,171.658165,171.158488,171.210018,170.688677,...,19.551164,25.344808,43.218177,78.790467,21.268197,14.789136,47.554547,61.493012,94.083156,116.029429


In [0]:
data.groupby(["category","difficulty"]).mean()

radius  dinamic_radius  radius_variance_X  \
category difficulty                                                  
numeric  easy        241.045810      142.498357          62.998198   
         hard        184.594773      148.714061          39.257791   
sequence easy        169.897227      142.003345          21.887426   
         hard        206.711836      143.637669          31.010925   
verbal   easy        182.938733      148.301190          42.973925   
         hard        204.491382      149.558088          48.230284   

                     radius_variance_Y  area_variance  
category difficulty                                    
numeric  easy                29.349955     514.474824  
         hard                21.016518     732.616399  
sequence easy                33.466933     583.006477  
         hard                22.721635     685.848629  
verbal   easy                23.227705     632.992135  
         hard                19.474251     505.198966

In [0]:
participant_id
category
difficulty
order
turn 
radius
dynamic_radius_ns
rounds_n_fseconds
rounds_n_lseconds
avg_distance_between_points
std_distance_between_points
displacement_center_n_seconds



In [0]:
data.columns

Index(['participant_id', 'category', 'difficulty', 'order', 'turn', 'radius',
       'dynamic_radius_30s', 'dynamic_radius_20s', 'dynamic_radius_15s',
       'dynamic_radius_10s', 'dynamic_radius_5s', 'rounds_10_fseconds',
       'rounds_15_fseconds', 'rounds_30_fseconds', 'rounds_60_fseconds',
       'rounds_10_lseconds', 'rounds_15_lseconds', 'rounds_30_lseconds',
       'rounds_60_lseconds', 'avg_distance_between_points',
       'std_distance_between_points', 'displacement_center_10_seconds',
       'displacement_center_15_seconds', 'displacement_center_30_seconds',
       'displacement_center_60_seconds'],
      dtype='object')

In [33]:
data

,participant_id,category,difficulty,order,turn,radius,dynamic_radius_30s,dynamic_radius_20s,dynamic_radius_15s,dynamic_radius_10s,dynamic_radius_5s,rounds_10_fseconds,rounds_15_fseconds,rounds_30_fseconds,rounds_60_fseconds,rounds_10_lseconds,rounds_15_lseconds,rounds_30_lseconds,rounds_60_lseconds,avg_distance_between_points,std_distance_between_points,distance_center_10_seconds,distance_center_15_seconds,distance_center_30_seconds,distance_center_60_seconds
0,p1,numeric,hard,1,R,218.320752,171.381115,170.996476,170.543908,170.397529,169.572365,17.711854,22.947926,41.240904,71.138766,18.328569,23.988042,40.261814,72.702734,20.61382,26.978427,29.008719,36.234571,79.219572,120.338036
